In [10]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [11]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='cora_ml',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_2',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [12]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [13]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [14]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([2995, 2995]) torch.Size([2995, 2995])
Training...
Epoch: 0001 train_loss= 1.38631 time= 0.05866
Epoch: 0002 train_loss= 1.38576 time= 0.06878
Epoch: 0003 train_loss= 1.38329 time= 0.07394
Epoch: 0004 train_loss= 1.37999 time= 0.07400
Epoch: 0005 train_loss= 1.37426 time= 0.07048
Epoch: 0006 train_loss= 1.36510 time= 0.07270
Epoch: 0007 train_loss= 1.35062 time= 0.07303
Epoch: 0008 train_loss= 1.33158 time= 0.07100
Epoch: 0009 train_loss= 1.30282 time= 0.07298
Epoch: 0010 train_loss= 1.26915 time= 0.07201
Epoch: 0011 train_loss= 1.23075 time= 0.07101
Epoch: 0012 train_loss= 1.19799 time= 0.07498
Epoch: 0013 train_loss= 1.16360 time= 0.07116
Epoch: 0014 train_loss= 1.15082 time= 0.07908
Epoch: 0015 train_loss= 1.12125 time= 0.08092
Epoch: 0016 train_loss= 1.09312 time= 0.07574
Epoch: 0017 train_loss= 1.08826 time= 0.07663
Epoch: 0018 train_loss= 1.06965 time= 0.08192
Epoch: 0019 train_loss= 1.06733 time= 0.07733
Epoch: 0020 train_loss= 1.05599 time= 0.07

Epoch: 0179 train_loss= 0.67042 time= 0.07503
Epoch: 0180 train_loss= 0.67759 time= 0.07300
Epoch: 0181 train_loss= 0.67169 time= 0.07600
Epoch: 0182 train_loss= 0.66696 time= 0.07708
Epoch: 0183 train_loss= 0.65907 time= 0.07237
Epoch: 0184 train_loss= 0.67975 time= 0.07202
Epoch: 0185 train_loss= 0.67100 time= 0.07351
Epoch: 0186 train_loss= 0.67398 time= 0.07890
Epoch: 0187 train_loss= 0.66504 time= 0.07300
Epoch: 0188 train_loss= 0.65706 time= 0.07100
Epoch: 0189 train_loss= 0.66985 time= 0.07200
Epoch: 0190 train_loss= 0.66665 time= 0.05375
Epoch: 0191 train_loss= 0.66583 time= 0.06738
Epoch: 0192 train_loss= 0.66291 time= 0.06550
Epoch: 0193 train_loss= 0.66893 time= 0.06812
Epoch: 0194 train_loss= 0.65979 time= 0.07640
Epoch: 0195 train_loss= 0.66581 time= 0.06403
Epoch: 0196 train_loss= 0.66174 time= 0.07262
Epoch: 0197 train_loss= 0.67655 time= 0.07201
Epoch: 0198 train_loss= 0.66134 time= 0.07085
Epoch: 0199 train_loss= 0.66230 time= 0.07439
Epoch: 0200 train_loss= 0.66330 ti

Epoch: 0155 train_loss= 0.87099 time= 0.07525
Epoch: 0156 train_loss= 0.86672 time= 0.06399
Epoch: 0157 train_loss= 0.85354 time= 0.06715
Epoch: 0158 train_loss= 0.86293 time= 0.06570
Epoch: 0159 train_loss= 0.87204 time= 0.08507
Epoch: 0160 train_loss= 0.87345 time= 0.07700
Epoch: 0161 train_loss= 0.86748 time= 0.07500
Epoch: 0162 train_loss= 0.85625 time= 0.07400
Epoch: 0163 train_loss= 0.86794 time= 0.07394
Epoch: 0164 train_loss= 0.85980 time= 0.07401
Epoch: 0165 train_loss= 0.88428 time= 0.07400
Epoch: 0166 train_loss= 0.86946 time= 0.07393
Epoch: 0167 train_loss= 0.89569 time= 0.06066
Epoch: 0168 train_loss= 0.86926 time= 0.06901
Epoch: 0169 train_loss= 0.86017 time= 0.07100
Epoch: 0170 train_loss= 0.86308 time= 0.06900
Epoch: 0171 train_loss= 0.87622 time= 0.07300
Epoch: 0172 train_loss= 0.87431 time= 0.07000
Epoch: 0173 train_loss= 0.85628 time= 0.07200
Epoch: 0174 train_loss= 0.85119 time= 0.07700
Epoch: 0175 train_loss= 0.86250 time= 0.07306
Epoch: 0176 train_loss= 0.86261 ti

Epoch: 0131 train_loss= 0.69410 time= 0.07594
Epoch: 0132 train_loss= 0.67180 time= 0.07371
Epoch: 0133 train_loss= 0.70287 time= 0.07221
Epoch: 0134 train_loss= 0.69514 time= 0.07286
Epoch: 0135 train_loss= 0.68780 time= 0.06601
Epoch: 0136 train_loss= 0.71120 time= 0.06998
Epoch: 0137 train_loss= 0.68418 time= 0.07326
Epoch: 0138 train_loss= 0.66963 time= 0.07504
Epoch: 0139 train_loss= 0.71869 time= 0.07119
Epoch: 0140 train_loss= 0.66371 time= 0.07156
Epoch: 0141 train_loss= 0.67522 time= 0.07296
Epoch: 0142 train_loss= 0.68417 time= 0.07160
Epoch: 0143 train_loss= 0.64499 time= 0.07398
Epoch: 0144 train_loss= 0.65516 time= 0.07078
Epoch: 0145 train_loss= 0.64038 time= 0.07100
Epoch: 0146 train_loss= 0.65140 time= 0.07499
Epoch: 0147 train_loss= 0.64700 time= 0.06215
Epoch: 0148 train_loss= 0.64196 time= 0.06904
Epoch: 0149 train_loss= 0.64901 time= 0.08100
Epoch: 0150 train_loss= 0.65159 time= 0.06676
Epoch: 0151 train_loss= 0.61645 time= 0.07276
Epoch: 0152 train_loss= 0.63574 ti

Epoch: 0109 train_loss= 0.85049 time= 0.07395
Epoch: 0110 train_loss= 0.85420 time= 0.07200
Epoch: 0111 train_loss= 0.84890 time= 0.07400
Epoch: 0112 train_loss= 0.83597 time= 0.06309
Epoch: 0113 train_loss= 0.81690 time= 0.06627
Epoch: 0114 train_loss= 0.83795 time= 0.08274
Epoch: 0115 train_loss= 0.84069 time= 0.06781
Epoch: 0116 train_loss= 0.83060 time= 0.07007
Epoch: 0117 train_loss= 0.82206 time= 0.07183
Epoch: 0118 train_loss= 0.82919 time= 0.06302
Epoch: 0119 train_loss= 0.83626 time= 0.06716
Epoch: 0120 train_loss= 0.82269 time= 0.06478
Epoch: 0121 train_loss= 0.82675 time= 0.07327
Epoch: 0122 train_loss= 0.81659 time= 0.07298
Epoch: 0123 train_loss= 0.81275 time= 0.06592
Epoch: 0124 train_loss= 0.81320 time= 0.06642
Epoch: 0125 train_loss= 0.82297 time= 0.07254
Epoch: 0126 train_loss= 0.80320 time= 0.07595
Epoch: 0127 train_loss= 0.80811 time= 0.07406
Epoch: 0128 train_loss= 0.80420 time= 0.07395
Epoch: 0129 train_loss= 0.79143 time= 0.06939
Epoch: 0130 train_loss= 0.79333 ti

...

Epoch: 0042 train_loss= 0.94965 time= 0.07598
Epoch: 0043 train_loss= 0.95879 time= 0.07402
Epoch: 0044 train_loss= 0.98376 time= 0.07100
Epoch: 0045 train_loss= 0.95404 time= 0.07201
Epoch: 0046 train_loss= 0.92558 time= 0.07134
Epoch: 0047 train_loss= 0.94631 time= 0.07098
Epoch: 0048 train_loss= 0.95348 time= 0.07191
Epoch: 0049 train_loss= 0.93808 time= 0.07087
Epoch: 0050 train_loss= 0.91786 time= 0.06901
Epoch: 0051 train_loss= 0.93881 time= 0.07199
Epoch: 0052 train_loss= 0.94661 time= 0.07506
Epoch: 0053 train_loss= 0.94742 time= 0.07101
Epoch: 0054 train_loss= 0.91643 time= 0.07193
Epoch: 0055 train_loss= 0.92339 time= 0.07400
Epoch: 0056 train_loss= 0.93342 time= 0.06900
Epoch: 0057 train_loss= 0.92699 time= 0.06900
Epoch: 0058 train_loss= 0.91359 time= 0.07200
Epoch: 0059 train_loss= 0.92024 time= 0.07318
Epoch: 0060 train_loss= 0.91767 time= 0.06900
Epoch: 0061 train_loss= 0.94026 time= 0.05985
Epoch: 0062 train_loss= 0.92780 time= 0.06229
Epoch: 0063 train_loss= 0.90760 ti

Epoch: 0019 train_loss= 1.07423 time= 0.06696
Epoch: 0020 train_loss= 1.06570 time= 0.06896
Epoch: 0021 train_loss= 1.04294 time= 0.07301
Epoch: 0022 train_loss= 1.04994 time= 0.06799
Epoch: 0023 train_loss= 1.02829 time= 0.07022
Epoch: 0024 train_loss= 1.00920 time= 0.07380
Epoch: 0025 train_loss= 1.01838 time= 0.06525
Epoch: 0026 train_loss= 0.99771 time= 0.07277
Epoch: 0027 train_loss= 1.00006 time= 0.06202
Epoch: 0028 train_loss= 0.99668 time= 0.08500
Epoch: 0029 train_loss= 1.00931 time= 0.07001
Epoch: 0030 train_loss= 0.98685 time= 0.07998
Epoch: 0031 train_loss= 0.97427 time= 0.07058
Epoch: 0032 train_loss= 0.99049 time= 0.07449
Epoch: 0033 train_loss= 0.99277 time= 0.07899
Epoch: 0034 train_loss= 0.96880 time= 0.06847
Epoch: 0035 train_loss= 0.95995 time= 0.06725
Epoch: 0036 train_loss= 0.96527 time= 0.06249
Epoch: 0037 train_loss= 0.95231 time= 0.06799
Epoch: 0038 train_loss= 0.94896 time= 0.08357
Epoch: 0039 train_loss= 0.95309 time= 0.06600
Epoch: 0040 train_loss= 0.97773 ti

Epoch: 0199 train_loss= 0.64267 time= 0.07800
Epoch: 0200 train_loss= 0.65124 time= 0.07400
Testing model...
Masking test edges...
torch.Size([2995, 2995]) torch.Size([2995, 2995])
Training...
Epoch: 0001 train_loss= 1.38642 time= 0.07700
Epoch: 0002 train_loss= 1.38566 time= 0.07308
Epoch: 0003 train_loss= 1.38433 time= 0.07600
Epoch: 0004 train_loss= 1.38194 time= 0.07300
Epoch: 0005 train_loss= 1.37806 time= 0.07400
Epoch: 0006 train_loss= 1.37138 time= 0.07400
Epoch: 0007 train_loss= 1.36059 time= 0.06600
Epoch: 0008 train_loss= 1.34511 time= 0.07275
Epoch: 0009 train_loss= 1.31970 time= 0.07500
Epoch: 0010 train_loss= 1.29165 time= 0.07300
Epoch: 0011 train_loss= 1.25331 time= 0.07600
Epoch: 0012 train_loss= 1.22164 time= 0.07500
Epoch: 0013 train_loss= 1.18206 time= 0.07500
Epoch: 0014 train_loss= 1.16152 time= 0.07300
Epoch: 0015 train_loss= 1.14680 time= 0.07085
Epoch: 0016 train_loss= 1.11936 time= 0.06987
Epoch: 0017 train_loss= 1.10857 time= 0.06444
Epoch: 0018 train_loss= 1

Epoch: 0176 train_loss= 0.62592 time= 0.07520
Epoch: 0177 train_loss= 0.64317 time= 0.06700
Epoch: 0178 train_loss= 0.64143 time= 0.07100
Epoch: 0179 train_loss= 0.67879 time= 0.07000
Epoch: 0180 train_loss= 0.71094 time= 0.07700
Epoch: 0181 train_loss= 0.79535 time= 0.07129
Epoch: 0182 train_loss= 0.63080 time= 0.07018
Epoch: 0183 train_loss= 0.68008 time= 0.06365
Epoch: 0184 train_loss= 0.78544 time= 0.06933
Epoch: 0185 train_loss= 0.61893 time= 0.08497
Epoch: 0186 train_loss= 0.78655 time= 0.07564
Epoch: 0187 train_loss= 0.74404 time= 0.07499
Epoch: 0188 train_loss= 0.70293 time= 0.07301
Epoch: 0189 train_loss= 0.74738 time= 0.07599
Epoch: 0190 train_loss= 0.61399 time= 0.07304
Epoch: 0191 train_loss= 0.71500 time= 0.07391
Epoch: 0192 train_loss= 0.63283 time= 0.07900
Epoch: 0193 train_loss= 0.68335 time= 0.07504
Epoch: 0194 train_loss= 0.62225 time= 0.07301
Epoch: 0195 train_loss= 0.66714 time= 0.07801
Epoch: 0196 train_loss= 0.62533 time= 0.07299
Epoch: 0197 train_loss= 0.63913 ti

Epoch: 0154 train_loss= 0.85540 time= 0.06934
Epoch: 0155 train_loss= 0.85515 time= 0.07567
Epoch: 0156 train_loss= 0.85066 time= 0.07499
Epoch: 0157 train_loss= 0.84031 time= 0.07401
Epoch: 0158 train_loss= 0.85060 time= 0.07198
Epoch: 0159 train_loss= 0.84013 time= 0.07501
Epoch: 0160 train_loss= 0.83767 time= 0.07499
Epoch: 0161 train_loss= 0.84102 time= 0.06500
Epoch: 0162 train_loss= 0.83707 time= 0.07400
Epoch: 0163 train_loss= 0.84306 time= 0.07300
Epoch: 0164 train_loss= 0.83950 time= 0.07400
Epoch: 0165 train_loss= 0.83782 time= 0.07659
Epoch: 0166 train_loss= 0.83305 time= 0.07441
Epoch: 0167 train_loss= 0.83054 time= 0.07286
Epoch: 0168 train_loss= 0.82823 time= 0.06434
Epoch: 0169 train_loss= 0.82489 time= 0.06571
Epoch: 0170 train_loss= 0.82311 time= 0.06874
Epoch: 0171 train_loss= 0.82392 time= 0.08577
Epoch: 0172 train_loss= 0.81531 time= 0.06695
Epoch: 0173 train_loss= 0.81905 time= 0.07200
Epoch: 0174 train_loss= 0.80809 time= 0.07300
Epoch: 0175 train_loss= 0.79404 ti